# Step 1
We import the .csv file of the Database of Art Historians filtered by nationality ('it') and manually integrated with Wikidata info. Then, we create a pandas dataframe to easily store and manipulate this information.

In [10]:
import pandas as pd
import re

# python3 -m pip install qwikidata
# python library for working with sparql and linked data from WikiData
from qwikidata.sparql import return_sparql_query_results

In [8]:
# create first dataframe only using the specified columns 
data = pd.read_csv("DoAH_StoriciItaliani_integrato.csv", sep=",",
                    usecols=["Full Name", "Gender", "Collection", "Keeper"], encoding="utf-8")

# axis 0 to drop the rows, subset to only remove NaNs from the column Archives
data.dropna(axis=0, subset=["Keeper"], inplace=True)

# resetting the index because all deleted rows have changed the length of the dataframe
data.reset_index(inplace=True, drop=True)

# .pickle is a python serialization format for easy and quick read-write, and pandas supports it natively
data.to_pickle("00_first_db.pickle")

# the first table we have looks like this:
pd.set_option("display.max_rows", None)
data.head(120)

,Full Name,Gender,Collection,Keeper
0,"Accascina, Maria",female,NaN,Comune di Palermo
1,"Agostini, Leonardo",male,NaN,Scuola Normale Superiore
2,"Alfieri, Vittorio",male,NaN,Biblioteca Medicea Laurenziana
3,"Alinari, Giuseppe",male,Archivio Alinari,Museo Nazionale Alinari della Fotografia
4,"Alinari, Leopoldo",male,Archivio Alinari,Museo Nazionale Alinari della Fotografia
5,"Arcangeli, Francesco",male,"Fondo speciale Angelo, Gaetano, Bianca e Franc...",Biblioteca comunale dell'Archiginnasio
6,"Aretino, Pietro",male,Fondo Bongi,State Archives of Lucca
7,"Argan, Giulio Carlo",male,NaN,Archivio privato a Roma
8,"Arias, Paolo Enrico",male,NaN,Scuola Normale Superiore
9,"Baglione, Giovanni",male,NaN,Archivio di Stato di Roma


# To resolve:
1. full names are reversed (`surname, name`)
2. we need to have a controlled entity (`wd:xyz`) for each name and keeper, to be able to link them to other info

In [9]:
def reformat_names(name):
    """ reverse names from surname,name format to name surname """
    l = name.split(", ")
    new = " ".join(reversed(l))
    # compile regex for multiple consecutive spaces
    return re.sub(r"\s+", " ", new)

In [10]:
# reverse names and remove duplicate whitespace
data["Full Name"] = data["Full Name"].apply(reformat_names)

data.describe()

,Full Name,Gender,Collection,Keeper
count,118,118,50,118
unique,118,2,49,77
top,Maria Accascina,male,Archivio Alinari,BEIC Digital Library
freq,1,108,2,10


In [11]:
historian_entity_from_label = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?artHistorian

WHERE {{
    ?artHistorian wdt:P31 wd:Q5 ;
                  wdt:P1412 ?language
                  FILTER (?language IN (wd:Q652, wd:Q397 ) ) 
    ?artHistorian wdt:P106 ?occupation
                  FILTER (?occupation IN (wd:Q1792450, wd:Q201788, wd:Q1622272, wd:Q3621491, wd:Q483501, wd:Q4164507, wd:Q4964182, wd:Q5697103, wd:Q33231 ) )    
    ?artHistorian rdfs:label ?o
                  FILTER ( str(?o) = "{}" )  .
}}
"""

In [12]:
def find_historian_entity_from_name(name: str):
    query = historian_entity_from_label.format(name)
    res = return_sparql_query_results(query_string=query)
    try:
        wdt_uri = res['results']['bindings'][0]['artHistorian']['value']
    except (IndexError, KeyError):
        return ""
    return wdt_uri.split("/")[-1]

In [13]:
data["Historian Entity"] = data["Full Name"].apply(find_historian_entity_from_name)

In [16]:
data.to_pickle("00_first_db.pickle")

In [14]:
pd.set_option("display.max_rows", None)
data.head(120)

,Full Name,Gender,Collection,Keeper,Historian Entity
0,Maria Accascina,female,NaN,Comune di Palermo,Q98804253
1,Leonardo Agostini,male,NaN,Scuola Normale Superiore,Q1054161
2,Vittorio Alfieri,male,NaN,Biblioteca Medicea Laurenziana,Q296244
3,Giuseppe Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,Q18934975
4,Leopoldo Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,Q16164590
5,Francesco Arcangeli,male,"Fondo speciale Angelo, Gaetano, Bianca e Franc...",Biblioteca comunale dell'Archiginnasio,Q1121086
6,Pietro Aretino,male,Fondo Bongi,State Archives of Lucca,
7,Giulio Carlo Argan,male,NaN,Archivio privato a Roma,Q778445
8,Paolo Enrico Arias,male,NaN,Scuola Normale Superiore,Q3894011
9,Giovanni Baglione,male,NaN,Archivio di Stato di Roma,Q983332


In [15]:
import numpy as np

print(data.replace(r'^\s*$', np.nan, regex=True).isnull().sum())

Full Name            0
Gender               0
Collection          68
Keeper               0
Historian Entity    13
dtype: int64


In [17]:
import pandas as pd
data = pd.read_pickle("00_first_db.pickle")
data.to_json("00_first_db.json")


# Step 3
___
### add a column with controlled entities for institutions with the role of "keeper"

In [1]:
import pandas as pd
from json import JSONDecodeError

In [61]:
data = pd.read_json("00_first_db.json")

In [62]:
keeper_entity_from_label = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?keeper

WHERE {{
    VALUES ?keeperRole {{wd:Q166118 wd:Q7075 wd:Q3953379 wd:Q3918 wd:Q33506 wd:Q17620767 wd:Q43229 wd:Q31855 wd:Q212805
                        wd:Q2352616 wd:Q1966910 wd:Q157031 wd:Q207694 wd:Q414147 wd:Q22806 wd:Q28564 wd:Q1329623 wd:Q44796387
                        wd:Q856234 wd:Q2122214 }}
    ?keeper wdt:P31 ?keeperRole .
    ?keeper rdfs:label ?o 
                  FILTER ( str(?o) = "{}" )  .
    
}}
"""

In [63]:
def find_keeper_entity_from_label(label: str):
    # remove trailing and leading whitespace
    label = label.strip()
    # substitute multiple spaces with a single one
    label = re.sub(r"\s+", " ", label)
    query = keeper_entity_from_label.format(label)
    try:
        res = return_sparql_query_results(query_string=query)
        wdt_uri = res['results']['bindings'][0]['keeper']['value']
    except (IndexError, KeyError, JSONDecodeError):
        return ""
    return wdt_uri.split("/")[-1]

In [64]:
def create_keeper_col(data):
    # create a new column computing the keeper entity from the keeper label
    data["Keeper Entity"] = data["Keeper"].apply(find_keeper_entity_from_label)

After gathering entities for art historians a problem we found was that the research was really slow (~35mins for the full dataframe lookup).
To speed things up this time we tried splitting the dataframe in two equal parts (around index 58) and launching two separate threads, each on a portion of the dataframe. This way, if the SPARQL engine takes a long time to respond, we have 2 concurrent calls being made: it reduced our running time to around 18 minutes for a full dataframe apply.

Since the approach seems to be successful we should probably try with 4/8 concurrent threads.

NB: this does not __speed up computation__, instead, when the operation is waiting for an IO task (waiting for sparql to respond with the JSON result for the query) it launches other requests or handle other responses without blocking.

In [65]:
# split df in half
df1 = data.iloc[:58, :]
df2 = data.iloc[58:, :]

# launch two threads running create_keeper_col on df1 and df2
from threading import Thread
t1 = Thread(target=create_keeper_col, args=(df1,))
t2 = Thread(target=create_keeper_col, args=(df2,))
t1.start()
t2.start()
# wait for the threads to finish
t1.join()
t2.join()

# concatenate the two dataframes
data = pd.concat([df1, df2], axis=0)

data.head(120)


<ipython-input-64-3cff0a060430>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Keeper Entity"] = data["Keeper"].apply(find_keeper_entity_from_label)


,Full Name,Gender,Collection,Keeper,Historian Entity,Keeper Entity
0,Maria Accascina,female,None,Comune di Palermo,Q98804253,Q81174665
1,Leonardo Agostini,male,None,Scuola Normale Superiore,Q1054161,Q672416
2,Vittorio Alfieri,male,None,Biblioteca Medicea Laurenziana,Q296244,Q856419
3,Giuseppe Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,Q18934975,Q1075580
4,Leopoldo Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,Q16164590,Q1075580
5,Francesco Arcangeli,male,"Fondo speciale Angelo, Gaetano, Bianca e Franc...",Biblioteca comunale dell'Archiginnasio,Q1121086,Q3639645
6,Pietro Aretino,male,Fondo Bongi,State Archives of Lucca,Q296272,Q3621654
7,Giulio Carlo Argan,male,None,Archivio privato a Roma,Q778445,
8,Paolo Enrico Arias,male,None,Scuola Normale Superiore,Q3894011,Q672416
9,Giovanni Baglione,male,None,Archivio di Stato di Roma,Q983332,Q2860424


In [66]:
data.to_pickle("01_second_db.pickle")

In [24]:
import pandas as pd
import numpy as np
with open("01_second_db.pickle", "rb") as fh:
    data = p.load(fh)
    
print(data.replace(r'^\s*$', np.nan, regex=True).isnull().sum())

Full Name            0
Gender               0
Collection          68
Keeper               0
Historian Entity     0
Keeper Entity       19
dtype: int64


In [19]:
data.to_json("01_second_db.json")

# Step 4
## Use wikidata entities to query relevant information about historians and keepers

After saving our new database in JSON format, we added and fill the missing keepers' entities manually. In some cases, we added __new items on Wikidata__ to give the organization a controlled entity. In the case of __private archives__, we decided to use the same entity (wd:Q12161242) to identify "archival collection or institution that is not accessible to the public".

Then, we created some queries to find out relevant information about historians and keepers to finalize our database: birth and death places and dates for historians, locations for keepers.

In [31]:
#Runna da qui
import pandas as pd
data = pd.read_json("01_second_db.json")
pd.set_option("display.max_rows", None)
data.head(120)

,Full Name,Gender,Collection,Keeper,Historian Entity,Keeper Entity
0,Maria Accascina,female,None,Comune di Palermo,Q98804253,Q81174665
1,Leonardo Agostini,male,None,Scuola Normale Superiore,Q1054161,Q672416
2,Vittorio Alfieri,male,None,Biblioteca Medicea Laurenziana,Q296244,Q856419
3,Giuseppe Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,Q18934975,Q1075580
4,Leopoldo Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,Q16164590,Q1075580
5,Francesco Arcangeli,male,"Fondo speciale Angelo, Gaetano, Bianca e Franc...",Biblioteca comunale dell'Archiginnasio,Q1121086,Q3639645
6,Pietro Aretino,male,Fondo Bongi,State Archives of Lucca,Q296272,Q3621654
7,Giulio Carlo Argan,male,None,Archivio privato a Roma,Q778445,Q12161242
8,Paolo Enrico Arias,male,None,Scuola Normale Superiore,Q3894011,Q672416
9,Giovanni Baglione,male,None,Archivio di Stato di Roma,Q983332,Q2860424


In [3]:
keepers_place_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?keeperPlace

WHERE {{
    ?keeper wdt:P131 ?keeperPlace .
    ?keeperPlace wdt:P460 ?o 
                FILTER ( str(?o) = "{}" )  .
}}
"""

In [ ]:
def find_keeperPlace_entity_from_entity(entity: str):
    # remove trailing and leading whitespace
    entity = entity.strip()
    # substitute multiple spaces with a single one
    entity = re.sub(r"\s+", " ", entity)
    query = keepers_place_query.format(entity)
    try:
        res = return_sparql_query_results(query_string=query)
        wdt_uri = res['results']['bindings'][0]['keeperPlace']['value']
    except (IndexError, KeyError, JSONDecodeError):
        return ""
    return wdt_uri.split("/")[-1]